# Create Mel-Spectrograms from flac audiofiles of  the VCTK dataset

## Setup

This notebook expects a zipped file containing the txt and wav48_silence_trimmed folders of VCTK in your google drive at the path  specified below.
Connect your google drive.

In [ ]:
vctk_gdrive_path = 'VC/VCTK/VCTK-Corpus-mic1.zip'
vctk_path = '/content/drive/MyDrive/' + vctk_gdrive_path

In [ ]:
!mkdir /content/vctk

copying and unzipping both take around 1 min.

In [ ]:
!cp $vctk_path /content/vctk/VCTK-Corpus-mic1.zip

In [ ]:
!unzip -q /content/vctk/VCTK-Corpus-mic1.zip -d /content/vctk

In [ ]:
!mkdir /content/vctk/mels

In [ ]:
vctk_path = '/content/vctk/'

### Requirements

In [ ]:
!pip install torch torchvision torchaudio

## Code

In [ ]:
import torch, torchaudio
import numpy as np
import os
from tqdm import tqdm
import torchaudio.transforms as transforms
import torch.nn.functional as F
from torchaudio.functional import resample

In [ ]:
class LogMelSpectrogram(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.melspctrogram = transforms.MelSpectrogram(
            sample_rate=16000,
            n_fft=1024,
            win_length=1024,
            hop_length=160,
            center=False,
            power=1.0,
            norm="slaney",
            onesided=True,
            n_mels=128,
            mel_scale="slaney",
        )

    def forward(self, wav):
        padding = (1024 - 160) // 2
        wav = F.pad(wav, (padding, padding), "reflect")
        mel = self.melspctrogram(wav)
        logmel = torch.log(torch.clamp(mel, min=1e-5))
        return logmel

In [ ]:
melspectrogram = LogMelSpectrogram()

In [ ]:
def load_flac(audiofile_path):
  data, sr = torchaudio.load(audiofile_path)
  data = resample(data, sr, 16000)
  return data

In [ ]:
def create_mel_spectrogram(audiofile_path, mel_path):
  data = load_flac(audiofile_path)
  data = data.unsqueeze(0)
  logmel = melspectrogram(data)
  logmel = logmel.squeeze().numpy()
  np.save(mel_path, logmel)

In [ ]:
spk_list = os.listdir(vctk_path + 'wav48_silence_trimmed')
spk_list.remove('log.txt')

In [ ]:
for spk in tqdm(spk_list):
  mel_spk_path = vctk_path + 'mels/' + spk
  audio_spk_path = vctk_path + 'wav48_silence_trimmed/' + spk
  os.makedirs(mel_spk_path, exist_ok=True)
  for audiofile in os.listdir(audio_spk_path):
    audiofile_path = audio_spk_path + '/' + audiofile
    mel_path = mel_spk_path + '/' + audiofile.split('.')[0] + '.npy'
    create_mel_spectrogram(audiofile_path, mel_path)


# Testing mel to audio

Let's test if the vocoder correctly generated audio from an example generated mel

In [ ]:
import IPython.display as display

In [ ]:
hifigan = torch.hub.load("bshall/hifigan:main", "hifigan_hubert_soft", trust_repo=True, map_location=torch.device('cpu'))

In [ ]:
mel = np.load('/content/vctk/mels/p228/p228_002_mic1.npy')
mel.shape

In [ ]:
mel = torch.Tensor(mel)
mel = mel.unsqueeze(0)
mel.shape

In [ ]:
with torch.inference_mode():
  target = hifigan(mel)

In [ ]:
display.Audio(target.squeeze(), rate=16000)

# Storing the results

the zipping takes about 10min.

In [ ]:
!zip -q -r /content/vctk/VCTK-Corpus-mic1-mels.zip /content/vctk/mels

In [ ]:
!cp /content/vctk/VCTK-Corpus-mic1-mels.zip /content/drive/MyDrive/VC/VCTK/VCTK-Corpus-mic1-mels.zip

In [ ]:
!du -sh /content/drive/MyDrive/VC/VCTK/*